In [2]:
import pandas as pd

df = pd.read_excel('FBI_dataset_cleaned.xlsx')

In [2]:
df.columns

Index(['TYPE', 'HUNDRED_BLOCK', 'NEIGHBOURHOOD', 'X', 'Y', 'Latitude',
       'Longitude', 'HOUR', 'MINUTE', 'YEAR', 'MONTH', 'DAY', 'Date'],
      dtype='object')

RANDOM FOREST

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

#  Prepare the data
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['YEAR'] = df['Date'].dt.year
df['MONTH'] = df['Date'].dt.month

# Group by year, month, and type
monthly_crime = df.groupby(['YEAR', 'MONTH', 'TYPE']).size().reset_index(name='Crime_Count')

#  Encode TYPE
monthly_crime['TYPE_CODE'] = monthly_crime['TYPE'].astype('category').cat.codes

# Save mapping from TYPE_CODE → TYPE
type_map = dict(enumerate(monthly_crime['TYPE'].astype('category').cat.categories))

#  Train model
X = monthly_crime[['YEAR', 'MONTH', 'TYPE_CODE']]
y = monthly_crime['Crime_Count']

model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X, y)

#  Prepare prediction dataframe
future_rows = []
for year in [2012, 2013]:
    for month in range(1, 13):
        for tcode in monthly_crime['TYPE_CODE'].unique():
            future_rows.append([year, month, tcode])

future_df = pd.DataFrame(future_rows, columns=['YEAR', 'MONTH', 'TYPE_CODE'])

# Keep TYPE names
future_df['TYPE'] = future_df['TYPE_CODE'].map(type_map)

# Predict
future_df['Crime_Count_Predicted'] = model.predict(future_df[['YEAR', 'MONTH', 'TYPE_CODE']])
future_df['Crime_Count_Predicted'] = future_df['Crime_Count_Predicted'].round().astype(int)


#  Clean output
final_df = future_df[['YEAR', 'MONTH', 'TYPE', 'Crime_Count_Predicted']]

print(final_df.head(20))
final_df.to_csv("fbi_predictions.csv", index=False)


    YEAR  MONTH                                               TYPE  \
0   2012      1                         Break and Enter Commercial   
1   2012      1                  Break and Enter Residential/Other   
2   2012      1                                           Mischief   
3   2012      1                           Offence Against a Person   
4   2012      1                                        Other Theft   
5   2012      1                                 Theft from Vehicle   
6   2012      1                                   Theft of Bicycle   
7   2012      1                                   Theft of Vehicle   
8   2012      1  Vehicle Collision or Pedestrian Struck (with I...   
9   2012      2                         Break and Enter Commercial   
10  2012      2                  Break and Enter Residential/Other   
11  2012      2                                           Mischief   
12  2012      2                           Offence Against a Person   
13  2012      2     

In [13]:
type_map

{0: 'Break and Enter Commercial',
 1: 'Break and Enter Residential/Other',
 2: 'Mischief',
 3: 'Offence Against a Person',
 4: 'Other Theft',
 5: 'Theft from Vehicle',
 6: 'Theft of Bicycle',
 7: 'Theft of Vehicle',
 8: 'Vehicle Collision or Pedestrian Struck (with Injury)'}

ACCURACY

In [11]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ==============================
# Prepare Data
# ==============================
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['YEAR_MONTH'] = df['Date'].dt.to_period('M')

all_months = pd.period_range(start=df['YEAR_MONTH'].min(), end=df['YEAR_MONTH'].max(), freq='M')
all_types = df['TYPE'].unique()

monthly_crime_type = df.groupby(['YEAR_MONTH', 'TYPE']).size().reset_index(name='Crime_Count')
all_combinations = pd.MultiIndex.from_product([all_months, all_types], names=['YEAR_MONTH', 'TYPE'])
monthly_crime_type = monthly_crime_type.set_index(['YEAR_MONTH', 'TYPE']).reindex(all_combinations, fill_value=0).reset_index()
monthly_crime_type = monthly_crime_type.sort_values(['TYPE', 'YEAR_MONTH']).reset_index(drop=True)

# ==============================
# Feature Engineering
# ==============================
feature_dfs = []
for crime_type in monthly_crime_type['TYPE'].unique():
    type_df = monthly_crime_type[monthly_crime_type['TYPE'] == crime_type].copy()

    for lag in [1, 2, 3, 6, 12]:
        type_df[f'Lag_{lag}'] = type_df['Crime_Count'].shift(lag)

    type_df['Rolling_3'] = type_df['Crime_Count'].shift(1).rolling(window=3).mean()
    type_df['Rolling_6'] = type_df['Crime_Count'].shift(1).rolling(window=6).mean()

    type_df['YEAR'] = type_df['YEAR_MONTH'].dt.year
    type_df['MONTH'] = type_df['YEAR_MONTH'].dt.month

    type_df = type_df.dropna().reset_index(drop=True)
    feature_dfs.append(type_df)

monthly_crime_features = pd.concat(feature_dfs, ignore_index=True)

# ==============================
# Accuracy Check
# ==============================
train_data = monthly_crime_features[monthly_crime_features['YEAR'] < 2011]
test_data = monthly_crime_features[monthly_crime_features['YEAR'] == 2011]

X_train = train_data.drop(['YEAR_MONTH', 'TYPE', 'Crime_Count'], axis=1)
y_train = train_data['Crime_Count']

X_test = test_data.drop(['YEAR_MONTH', 'TYPE', 'Crime_Count'], axis=1)
y_test = test_data['Crime_Count']

model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Accuracy Check (2011 test data):")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

# ==============================
# Forecast for 2012 and 2013
# ==============================
future_predictions = []

for crime_type in monthly_crime_features['TYPE'].unique():
    type_data = monthly_crime_features[monthly_crime_features['TYPE'] == crime_type].copy()

    X_train_type = type_data.drop(['YEAR_MONTH', 'TYPE', 'Crime_Count'], axis=1)
    y_train_type = type_data['Crime_Count']

    model = RandomForestRegressor(n_estimators=200, random_state=42)
    model.fit(X_train_type, y_train_type)

    last_known = type_data.iloc[-1].copy()

    for i in range(24):  # forecast for 24 months
        year_month = last_known['YEAR_MONTH'] + 1

        new_row = {
            "YEAR_MONTH": year_month,
            "TYPE": crime_type,
            "YEAR": year_month.year,
            "MONTH": year_month.month
        }

        # Lags
        for lag in [1, 2, 3, 6, 12]:
            col = f'Lag_{lag}'
            if lag == 1:
                new_row[col] = last_known['Crime_Count']
            else:
                prev_index = type_data[type_data['YEAR_MONTH'] == year_month - lag]
                if not prev_index.empty:
                    new_row[col] = prev_index.iloc[0]['Crime_Count']
                else:
                    new_row[col] = last_known['Crime_Count']

        # Rolling averages
        new_row['Rolling_3'] = np.mean([new_row[f'Lag_{l}'] for l in [1,2,3]])
        new_row['Rolling_6'] = np.mean([new_row[f'Lag_{l}'] for l in [1,2,3,6]])

        # Build prediction row exactly matching X_train columns
        X_new = pd.DataFrame([{col: new_row[col] for col in X_train_type.columns}])

        pred = model.predict(X_new)[0]
        new_row['Crime_Count'] = int(round(pred))

        future_predictions.append(new_row)

        # Update last_known for next iteration
        last_known = pd.Series(new_row)
        type_data = pd.concat([type_data, pd.DataFrame([new_row])], ignore_index=True)

# ==============================
# Output Forecast
# ==============================
future_df = pd.DataFrame(future_predictions)
future_df = future_df[['YEAR', 'MONTH', 'TYPE', 'Crime_Count']]

print(future_df.head(20))
future_df.to_csv("fbi_forecast_with_accuracy.csv", index=False)


Accuracy Check (2011 test data):
MAE: 33.82
RMSE: 59.60
R² Score: 0.94
    YEAR  MONTH                        TYPE  Crime_Count
0   2012      1  Break and Enter Commercial          215
1   2012      2  Break and Enter Commercial          200
2   2012      3  Break and Enter Commercial          197
3   2012      4  Break and Enter Commercial          200
4   2012      5  Break and Enter Commercial          195
5   2012      6  Break and Enter Commercial          204
6   2012      7  Break and Enter Commercial          199
7   2012      8  Break and Enter Commercial          205
8   2012      9  Break and Enter Commercial          198
9   2012     10  Break and Enter Commercial          206
10  2012     11  Break and Enter Commercial          200
11  2012     12  Break and Enter Commercial          211
12  2013      1  Break and Enter Commercial          202
13  2013      2  Break and Enter Commercial          198
14  2013      3  Break and Enter Commercial          196
15  2013      4  

In [22]:
type_df

,YEAR_MONTH,TYPE,Crime_Count,Lag_1,Lag_2,Lag_3,Lag_6,Lag_12,Lag_24,Rolling_3,Rolling_6,YEAR,MONTH,Month_sin,Month_cos
0,2001-01,Vehicle Collision or Pedestrian Struck (with I...,108,174.0,178.0,202.0,155.0,131.0,139.0,184.666667,174.000000,2001,1,5.000000e-01,8.660254e-01
1,2001-02,Vehicle Collision or Pedestrian Struck (with I...,135,108.0,174.0,178.0,173.0,121.0,103.0,153.333333,166.166667,2001,2,8.660254e-01,5.000000e-01
2,2001-03,Vehicle Collision or Pedestrian Struck (with I...,163,135.0,108.0,174.0,162.0,151.0,127.0,139.000000,159.833333,2001,3,1.000000e+00,6.123234e-17
3,2001-04,Vehicle Collision or Pedestrian Struck (with I...,142,163.0,135.0,108.0,202.0,114.0,141.0,135.333333,160.000000,2001,4,8.660254e-01,-5.000000e-01
4,2001-05,Vehicle Collision or Pedestrian Struck (with I...,181,142.0,163.0,135.0,178.0,160.0,158.0,146.666667,150.000000,2001,5,5.000000e-01,-8.660254e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,2011-08,Vehicle Collision or Pedestrian Struck (with I...,148,149.0,163.0,144.0,122.0,130.0,131.0,152.000000,140.333333,2011,8,-8.660254e-01,-5.000000e-01
128,2011-09,Vehicle Collision or Pedestrian Struck (with I...,142,148.0,149.0,163.0,130.0,160.0,128.0,153.333333,144.666667,2011,9,-1.000000e+00,-1.836970e-16
129,2011-10,Vehicle Collision or Pedestrian Struck (with I...,134,142.0,148.0,149.0,134.0,141.0,154.0,146.333333,146.666667,2011,10,-8.660254e-01,5.000000e-01
130,2011-11,Vehicle Collision or Pedestrian Struck (with I...,158,134.0,142.0,148.0,144.0,140.0,110.0,141.333333,146.666667,2011,11,-5.000000e-01,8.660254e-01


In [23]:
monthly_crime_features

,YEAR_MONTH,TYPE,Crime_Count,Lag_1,Lag_2,Lag_3,Lag_6,Lag_12,Lag_24,Rolling_3,Rolling_6,YEAR,MONTH,Month_sin,Month_cos
0,2001-01,Break and Enter Commercial,193,265.0,271.0,318.0,237.0,216.0,303.0,284.666667,289.166667,2001,1,5.000000e-01,8.660254e-01
1,2001-02,Break and Enter Commercial,211,193.0,265.0,271.0,266.0,256.0,254.0,243.000000,281.833333,2001,2,8.660254e-01,5.000000e-01
2,2001-03,Break and Enter Commercial,204,211.0,193.0,265.0,378.0,275.0,292.0,223.000000,272.666667,2001,3,1.000000e+00,6.123234e-17
3,2001-04,Break and Enter Commercial,154,204.0,211.0,193.0,318.0,289.0,266.0,202.666667,243.666667,2001,4,8.660254e-01,-5.000000e-01
4,2001-05,Break and Enter Commercial,201,154.0,204.0,211.0,271.0,254.0,291.0,189.666667,216.333333,2001,5,5.000000e-01,-8.660254e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183,2011-08,Vehicle Collision or Pedestrian Struck (with I...,148,149.0,163.0,144.0,122.0,130.0,131.0,152.000000,140.333333,2011,8,-8.660254e-01,-5.000000e-01
1184,2011-09,Vehicle Collision or Pedestrian Struck (with I...,142,148.0,149.0,163.0,130.0,160.0,128.0,153.333333,144.666667,2011,9,-1.000000e+00,-1.836970e-16
1185,2011-10,Vehicle Collision or Pedestrian Struck (with I...,134,142.0,148.0,149.0,134.0,141.0,154.0,146.333333,146.666667,2011,10,-8.660254e-01,5.000000e-01
1186,2011-11,Vehicle Collision or Pedestrian Struck (with I...,158,134.0,142.0,148.0,144.0,140.0,110.0,141.333333,146.666667,2011,11,-5.000000e-01,8.660254e-01
